# NPP PROBLEM

The Number Partitioning Problem is the problem where, given a set of Integer, you are trying to find the two subset such that the sum of elements in the first subset is equal to the sum of elements in the second subset. Formally it should be:
$$
S = \{s_i | s_i \in \mathbb{N}\}
$$
we want to find 
$$
A \subset S, B = S-A
$$
such that
$$
\sum_{a \in A} a = \sum_{b \in B} b
$$
if there are no such subset, the solution should be A, B such that the difference
$$
(\sum_{a \in A} a - \sum_{b \in B} b)^2
$$
is minimized. 

Since finding random feasible solution is pretty easy and since performing a sample from the neighborhood is pretty easy too, this lead us to approach the problem with a Simulated Annealing approach.

### Guaranteed solution

For the first part of this notebook we assume there always exists a solution for the exact problem (we do not minimize the difference, we can impose it to 0).

In [1]:
# Function to generate a Set with guaranteed subsets with equal sum
import random
A = [random.randint(1, 100) for i in range(10)]
B = []
while (sum(B) < sum(A)):
    B.append(random.randint(1, 100))
diff = sum(B) - sum(A)
A[0] += diff

S = A + B
random.shuffle(S)

Usually we could like to implement an heuristic to find the starting point of our simulaed annealing, btw for the first parte we do not use any heuristic in order to obtain it.

In [2]:
def random_subsets(S): 
    A = []
    B = []
    for s in S:
        u = random.uniform(0, 1)
        if u > 0.5:
            A.append(s)
        else:
            B.append(s)
    return A, B

Also while doing sampling within the neighborhood of our solution we could use some heuristic to help the algorithm in finding possibly "better" solution, but for the first part we do not implement any heuristic in order to obtain it.

In [3]:
def sample_from_neighborhood(A, B, n_mut = 2): #  The number of mutation can be tweaked based on the value of the solution
    n_mut_A = random.randrange(0, min(n_mut, len(A)))
    A_index = [i for i in range (len(A))]
    random.shuffle(A_index)
    selected_indexes_A = A_index[:n_mut_A]
    n_mut_B = random.randrange(0, min(n_mut, len(B)))
    B_index = [i for i in range (len(B))]
    random.shuffle(B_index)
    selected_indexes_B = B_index[:n_mut_B]
    for i in selected_indexes_A:
        B.append(A[i])
    for i in selected_indexes_B:
        A.append(B[i])
    return A, B

The evaluation function is just the square of the difference of the sum of the subset.

In [5]:
def evaluate_solution(A, B):
    return (sum(A) - sum(B))**2

The acceptance probability is computed as protocol in a SA.
$$
p = \exp(-\frac{\Delta{E}}{T})
$$

In [4]:
import math
def acceptance_probability(e_old, e_new, T):
    
    delta_e = e_new - e_old
    if delta_e > 0:
        return 1
    if -delta_e/T > 700:
        return 0
    return math.exp(-(delta_e/T))

Here we define a geometric schedule of the temperature.

In [7]:
def geometric_schedule(T, **kwargs):
    k = kwargs["k"]
    return k*T

We define a generic SA cycle.

In [6]:
def simulated_annealing_cycle(initial_solution, T_0, schedule, **kwargs):
    T = T_0
    schedule_params = {key:item for key, item in kwargs.items()}
    solution = initial_solution
    while T > 1:
        A_old, B_old = solution
        E_old = evaluate_solution(A_old, B_old)

        if E_old == 0:
            return solution
        
        A_new, B_new = sample_from_neighborhood(A_old, B_old)
        E_new = evaluate_solution(A_new, B_new)
        
        p = acceptance_probability(E_old, E_new, T)
        if random.uniform(0, 1) <= p:
            solution = [A_new, B_new]

        T = schedule(T, **schedule_params)
        
    return solution

And we try now how it works.

In [10]:
A, B = random_subsets(S)
A, B = simulated_annealing_cycle([A, B], 10000, geometric_schedule, k = 0.995)
print(sum(A))
print(sum(B))

29738
29738


This solution works enough but both k and T are pretty high, btw it's pretty fast.

### Solution not Guaranteed and Heuristics

This part is dedicated to the solution at the problem when we want to minimize the difference (and not imposing it to 0). In this part we will also try some heuristics.